## SVM Model Hyperparameter Tuning

### Library and Data Importation

In [1]:
#import sys
#!{sys.executable} -m pip install cvxpy
import cvxpy as cp
import pandas as pd
import numpy as np
from numpy import array
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
import matplotlib.pyplot as plt

#import data set
nfl1 = pd.read_csv("NFL_data_super_cleaned.csv")

nfl1.dropna(inplace=True)
display(nfl1)
print(nfl1.isnull().values.any(), nfl1.isnull().sum().sum())

,posteam,yardline_100,quarter_seconds_remaining,qtr,down,goal_to_go,ydstogo,play_type,score_margin
0,PIT,58.0,893.0,1,1.0,0.0,10,pass,0.0
1,PIT,53.0,856.0,1,2.0,0.0,5,run,0.0
2,PIT,56.0,815.0,1,3.0,0.0,8,pass,0.0
3,PIT,56.0,807.0,1,4.0,0.0,8,kick,0.0
4,TEN,98.0,796.0,1,1.0,0.0,10,run,0.0
...,...,...,...,...,...,...,...,...,...
353055,CAR,71.0,82.0,4,2.0,0.0,1,pass,-5.0
353056,CAR,71.0,77.0,4,3.0,0.0,1,pass,-5.0
353057,CAR,66.0,63.0,4,2.0,0.0,10,pass,-5.0
353058,CAR,66.0,58.0,4,3.0,0.0,10,pass,-5.0


False 0


### Data Preprocessing

In [2]:
%%time
#Our 350,000 samples seem like a little too much, so sample about 10,000 rows
sample = nfl1.sample(n=25000, random_state=21, axis=0)

#y needs to be one-dimensional for SVM, so don't encode play_type
labels = np.array(sample["play_type"])
sample = sample.drop("play_type", axis=1)
#one-hot encode categorical feature variables
sample = pd.get_dummies(sample)
#feature_list = list(sample.columns)
features = np.array(sample)
display(sample)
print(sample.isnull().values.any(), sample.isnull().sum().sum())

#split data into training and testing sets
#seed: 21, train/test ratio: 0.2 test, 0.8 train
X_train, X_test, y_train, y_test = train_test_split(sample.to_numpy(), labels, test_size = 0.2, random_state = 21)
#compares each play_type to the other possible play_type

scaler = preprocessing.StandardScaler()
scaler.fit(X_train)

clf_ovo = SVC(kernel='rbf', decision_function_shape='ovo') # The other is ovr

clf_ovo.fit(scaler.transform(X_train), np.asarray(y_train))

print(classification_report(y_test, clf_ovo.predict(scaler.transform(X_test))))


,yardline_100,quarter_seconds_remaining,qtr,down,goal_to_go,ydstogo,score_margin,posteam_ARI,posteam_ATL,posteam_BAL,...,posteam_NO,posteam_NYG,posteam_NYJ,posteam_PHI,posteam_PIT,posteam_SEA,posteam_SF,posteam_TB,posteam_TEN,posteam_WAS
222356,61.0,119.0,1,3.0,0.0,22,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
61840,78.0,702.0,1,4.0,0.0,4,-7.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2684,61.0,585.0,4,2.0,0.0,4,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
225580,73.0,483.0,3,3.0,0.0,3,-3.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46520,70.0,201.0,1,1.0,0.0,10,7.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67126,42.0,532.0,3,1.0,0.0,10,-7.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
136191,42.0,659.0,4,1.0,0.0,10,-31.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
81209,66.0,22.0,1,1.0,0.0,10,-7.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
322811,69.0,596.0,1,2.0,0.0,8,-7.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


False 0
              precision    recall  f1-score   support

        kick       0.88      0.96      0.92       463
        pass       0.68      0.77      0.72      2633
         run       0.62      0.49      0.54      1904

    accuracy                           0.68      5000
   macro avg       0.73      0.74      0.73      5000
weighted avg       0.67      0.68      0.67      5000

Wall time: 52.6 s


### SVM RBF Kernel Tuning

In [3]:
%%time

# defining parameter range
param_grid = {'C': [0.5, 1, 1.5],
              'gamma': [0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
 
grid = GridSearchCV(SVC(), param_grid, refit = True, cv = 3, n_jobs=1, verbose = 4, scoring="accuracy")
 
# fitting the model for grid search
grid.fit(X_train, y_train)
grid_predictions = grid.predict(X_test)
print(grid.best_estimator_)

print(classification_report(y_test,grid_predictions))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END ......C=0.5, gamma=0.1, kernel=rbf;, score=0.530 total time= 2.0min
[CV 2/3] END ......C=0.5, gamma=0.1, kernel=rbf;, score=0.530 total time= 2.0min
[CV 3/3] END ......C=0.5, gamma=0.1, kernel=rbf;, score=0.532 total time= 1.9min
[CV 1/3] END .....C=0.5, gamma=0.01, kernel=rbf;, score=0.537 total time= 1.0min
[CV 2/3] END .....C=0.5, gamma=0.01, kernel=rbf;, score=0.538 total time=  59.6s
[CV 3/3] END .....C=0.5, gamma=0.01, kernel=rbf;, score=0.528 total time= 1.0min
[CV 1/3] END ....C=0.5, gamma=0.001, kernel=rbf;, score=0.547 total time=  37.6s
[CV 2/3] END ....C=0.5, gamma=0.001, kernel=rbf;, score=0.545 total time=  37.5s
[CV 3/3] END ....C=0.5, gamma=0.001, kernel=rbf;, score=0.545 total time=  39.6s
[CV 1/3] END ...C=0.5, gamma=0.0001, kernel=rbf;, score=0.537 total time=  35.9s
[CV 2/3] END ...C=0.5, gamma=0.0001, kernel=rbf;, score=0.538 total time=  35.0s
[CV 3/3] END ...C=0.5, gamma=0.0001, kernel=rbf;

### SVM Linear Kernel Tuning

In [11]:
%%time

# defining parameter range
param_grid = {'C': [0.5, 1, 1.5],
              'kernel': ['linear']}
 
grid = GridSearchCV(SVC(), param_grid, refit = True, cv = 3, n_jobs=1, verbose = 4, scoring="accuracy")
 
# fitting the model for grid search
grid.fit(X_train, y_train)
grid_predictions = grid.predict(X_test)
print(grid.best_estimator_)

print(classification_report(y_test,grid_predictions))

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV 1/3] END ..............C=0.5, kernel=linear;, score=0.679 total time=11.3min
[CV 2/3] END ..............C=0.5, kernel=linear;, score=0.673 total time=11.5min
[CV 3/3] END ..............C=0.5, kernel=linear;, score=0.673 total time=13.3min
[CV 1/3] END ................C=1, kernel=linear;, score=0.678 total time=19.9min
[CV 2/3] END ................C=1, kernel=linear;, score=0.673 total time=20.7min
[CV 3/3] END ................C=1, kernel=linear;, score=0.673 total time=23.3min
[CV 1/3] END ..............C=1.5, kernel=linear;, score=0.679 total time=32.5min
[CV 2/3] END ..............C=1.5, kernel=linear;, score=0.672 total time=28.3min
[CV 3/3] END ..............C=1.5, kernel=linear;, score=0.673 total time=29.6min
SVC(C=0.5, kernel='linear')
              precision    recall  f1-score   support

        kick       0.85      0.97      0.91       274
        pass       0.70      0.74      0.72      1614
         run       0.